In [14]:
import os, shutil
import random

from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Model

from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception

In [2]:
cwd_dir = "./drive/MyDrive/Python Code/kaggle_bee_vs_wasp"
# os.chdir(cwd_dir)


train_dir = os.path.join(cwd_dir, 'data', 'train')
val_dir = os.path.join(cwd_dir, 'data', 'validation')
test_dir = os.path.join(cwd_dir, 'data', 'test')
# os.mkdir(train_dir)
# os.mkdir(val_dir)
# os.mkdir(test_dir)

# train_bee_dir = os.path.join(train_dir, 'bee')
# train_wasp_dir = os.path.join(train_dir, 'wasp')
# train_insect_dir = os.path.join(train_dir, 'insect')
# train_other_dir = os.path.join(train_dir, 'other')
# os.mkdir(train_bee_dir)
# os.mkdir(train_wasp_dir)
# os.mkdir(train_insect_dir)
# os.mkdir(train_other_dir)

# val_bee_dir = os.path.join(val_dir, 'bee')
# val_wasp_dir = os.path.join(val_dir, 'wasp')
# val_insect_dir = os.path.join(val_dir, 'insect')
# val_other_dir = os.path.join(val_dir, 'other')
# os.mkdir(val_bee_dir)
# os.mkdir(val_wasp_dir)
# os.mkdir(val_insect_dir)
# os.mkdir(val_other_dir)

# test_bee_dir = os.path.join(test_dir, 'bee')
# test_wasp_dir = os.path.join(test_dir, 'wasp')
# test_insect_dir = os.path.join(test_dir, 'insect')
# test_other_dir = os.path.join(test_dir, 'other')
# os.mkdir(test_bee_dir)
# os.mkdir(test_wasp_dir)
# os.mkdir(test_insect_dir)
# os.mkdir(test_other_dir)

# dict_folders = {'bee' : 'bee1', 'wasp' : 'wasp1', 'insect' : 'other_insect', 'other' : 'other_noinsect'}

# random.seed(1234)
# for i in range(0, len(dict_folders)):
#   src_folder_chosen = list(dict_folders.values())[i]
#   src_path_chosen = os.path.join(cwd_dir, src_folder_chosen)
#   dst_folder_chosen = list(dict_folders.keys())[i]
#   # dst_path_chosen = os.path.join(dst_folder_chosen)

#   file_names = [f for f in os.listdir(src_path_chosen) if os.path.isfile(os.path.join(src_path_chosen, f))]
#   for file_name in file_names:
#     src = os.path.join(src_path_chosen, file_name)
#     n_random = random.uniform(0, 1)
#     if n_random < 0.65:
#       dst = os.path.join(train_dir, dst_folder_chosen, file_name)
#     elif n_random < 0.85:
#       dst = os.path.join(val_dir, dst_folder_chosen, file_name)
#     elif n_random <= 1:
#       dst = os.path.join(test_dir, dst_folder_chosen, file_name)
#     shutil.copyfile(src, dst)


In [3]:
SHAPE = 224
BATCH_SIZE = 16

In [4]:
# model from scratch
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (SHAPE, SHAPE, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(4, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.RMSprop(lr = 0.00001), metrics = ['acc'])

earlystop = EarlyStopping(monitor = 'val_loss', patience = 4, verbose = 1)

history = model.fit_generator(
    train_generator,
    epochs = 30,
    callbacks = earlystop,
    validation_data = valid_generator
)
# Save our model for inference
# model.save("model-weights/xception.h5")
model.save("primo.h5")

In [8]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    horizontal_flip = True
)
valid_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 0,
    width_shift_range = 0.0,
    height_shift_range = 0.0,
    horizontal_flip = False
)
train_generator = train_datagen.flow_from_directory(
    os.path.join(cwd_dir, 'data', 'train/'),
    target_size = (SHAPE, SHAPE),
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
)
valid_generator = valid_datagen.flow_from_directory(
    os.path.join(cwd_dir, 'data', 'validation/'),
    target_size = (SHAPE, SHAPE),
    shuffle = True,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
)


Found 5126 images belonging to 4 classes.
Found 1562 images belonging to 4 classes.


In [16]:
# pre-trained model
model = Xception(input_shape = (SHAPE, SHAPE, 3), include_top = False, weights = 'imagenet')

x = model.output
x = layers.AveragePooling2D(pool_size = (2, 2))(x)
x = layers.Dense(32, activation = 'relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Flatten()(x)
x = layers.Dense(4, activation = 'softmax', kernel_regularizer = l2(.0005))(x)
model = Model(inputs = model.inputs, outputs = x)
opt = optimizers.SGD(lr = 0.0001, momentum = .9)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

earlystop = EarlyStopping(monitor = 'val_loss', patience = 4, verbose = 1)
checkpoint = ModelCheckpoint(
    os.path.join(cwd_dir, "model-weights/xception_checkpoint.h5"),
    monitor = "val_loss",
    mode = "min",
    save_best_only = True,
    verbose = 1
)

history = model.fit_generator(
    train_generator,
    epochs = 20,
    callbacks = [earlystop, checkpoint],
    validation_data = valid_generator
)
# Save our model for inference
model.save(os.path.join(cwd_dir, "model-weights/xception.h5"), save_format = 'h5')



/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
321/321 [==============================] - 114s 341ms/step - loss: 1.2803 - accuracy: 0.3997 - val_loss: 0.8384 - val_accuracy: 0.6588

Epoch 00001: val_loss improved from inf to 0.83839, saving model to ./drive/MyDrive/Python Code/kaggle_bee_vs_wasp/model-weights/xception_checkpoint.h5
Epoch 2/20
321/321 [==============================] - 111s 345ms/step - loss: 0.8414 - accuracy: 0.6684 - val_loss: 0.6295 - val_accuracy: 0.7465

Epoch 00002: val_loss improved from 0.83839 to 0.62948, saving model to ./drive/MyDrive/Python Code/kaggle_bee_vs_wasp/model-weights/xception_checkpoint.h5
Epoch 3/20
321/321 [==============================] - 112s 347ms/step - loss: 0.6743 - accuracy: 0.7429 - val_loss: 0.5260 - val_accuracy: 0.8035

Epoch 00003: val_loss improved from 0.62948 to 0.52595, saving model to ./drive/MyDrive/Python Code/kaggle_bee_vs_wasp/model-weights/xception_checkpoint.h5
Epoch 4/20
321/321 [==============================] - 112s 347ms/step - loss: 0.5829 - accuracy

In [17]:
model.save(os.path.join(cwd_dir, "model-weights/xception.h5"), save_format = 'h5')